In [7]:
import pickle
from collections import OrderedDict
from pathlib import Path

import numpy as np
from matplotlib import pyplot as plt
from periodictable import elements
from tqdm import tqdm

In [8]:
from exfor_tools.distribution import AngularDistribution

In [9]:
from jitr.optical_potentials import kduq, wlh

In [10]:
import rxmc

In [11]:
import elm

In [22]:
results_dir = Path("./simple_so_no_intercept")

In [23]:
neutron = (1, 0)
proton = (1, 1)

## Read in samples

In [20]:
def draw_samples(chains, draws, burn):
    return chains[:, burn:, :].reshape(
        chains.shape[0] * (chains.shape[1] - burn), chains.shape[2]
    )[np.random.choice(chains.shape[0] * (chains.shape[1] - burn), draws), :]

In [24]:
posterior_samples = draw_samples(np.load(results_dir / "all_chains.npy"), 1000, 10000)

## read in corpora

In [25]:
with open("./combined_elastic_corpus.pkl", "rb") as f:
    corpus = pickle.load(f)

ModuleNotFoundError: No module named 'elm.corpus'

## Propagate ELM prior into constraint observables

In [14]:
yth_nn = []
for i, constraint in enumerate(tqdm(corpus.constraints)):
    yth_nn.append(np.vstack([constraint.model(p) for p in posterior_samples]))

  0%|                                                                                                                                     | 0/2 [00:00<?, ?it/s]


AttributeError: 'ElasticAngularCorpus' object has no attribute 'model'

## Write posterior predictive distributions to disk

In [15]:
with open(output_dir / "nn_elm_prior_predictive.pkl", "wb") as f:
    pickle.dump(yth_nn, f)

FileNotFoundError: [Errno 2] No such file or directory: 'elm_posterior_propagation/nn_elm_prior_predictive.pkl'

In [14]:
with open(output_dir / "pp_elm_prior_predictive.pkl", "wb") as f:
    pickle.dump(yth_pp, f)